In [ ]:
!pip install aicsimageio[all] --user
!pip install bioformats_jar
!pip install aicspylibczi>=3.1.1 fsspec>=2022.8.0
!pip install opencv-python

In [ ]:
from aicsimageio.writers import OmeTiffWriter
from aicsimageio import AICSImage
from skimage.io import imshow
from tqdm import tqdm
import os
import cv2

import numpy as np
from aicspylibczi import CziFile
from tqdm import tqdm
import pandas as pd

In [ ]:
def process_czi_file(czi_file_path, output_folder, subpart_size):
    df = pd.DataFrame(columns=('X', 'Y', 'Z', 'focus'))
        
    # Create a CziFile object
    czi = CziFile(czi_file_path)

    df = pd.DataFrame(columns=['X','Y','Z','focus'])
    
    # Get the shape of the data
    size = czi.size
    
    print(czi.dims)
    print(size)

    # Check if the file is a mosaic
    is_mosaic = czi.is_mosaic()

    if not is_mosaic:
        print("The provided CZI file is not a mosaic.")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    mosaic_data_shape = czi.read_mosaic(C=0, scale_factor=1, Z=0).shape
    num_subparts_x = mosaic_data_shape[2] // subpart_size[0]
    num_subparts_y = mosaic_data_shape[3] // subpart_size[1]
    total_iterations = num_subparts_x * num_subparts_y * size[2]
    
    print(f"Total integrations: X {num_subparts_x}, Y {num_subparts_y}, Z {size[2]}")

    progress_bar = tqdm(total=total_iterations, desc=f"Processing {os.path.basename(czi_file_path)}")

    for z in range(size[2]):
        mosaic_data = czi.read_mosaic(C=0, scale_factor=1, Z=z)

        for x in range(num_subparts_x):
            for y in range(num_subparts_y):
                subpart = mosaic_data[:, :, x * subpart_size[0]:(x + 1) * subpart_size[0],
                                      y * subpart_size[1]:(y + 1) * subpart_size[1], :]

                aics_image = AICSImage(subpart)

                output_file = os.path.join(output_folder, f"subpart_z-{z}_x-{x}_y-{y}.ome.tiff")
                
                focus = cv2.Laplacian(np.squeeze(subpart), cv2.CV_64F, ksize=5).var()
                # Append Row to DataFrame
                list_row = {"X": x, "Y": y, "Z": z, "focus": focus}
                df.loc[len(df)]= list_row

                OmeTiffWriter.save(cv2.cvtColor(np.squeeze(subpart), cv2.COLOR_BGR2RGB), output_file, dim_order="YXS")

                progress_bar.update(1)

    progress_bar.close()
    output_csv = os.path.join(output_folder, f"_focus_estimate.csv")
    df.to_csv(output_csv)


In [ ]:
%%time
# czi_file_path = '../../230518_ML_pyl_OB/2023_05_21_Pyl-OB_001.czi'
# name = "2023_05_21_Pyl-OB_001"

# test
# czi_file_path = '../../test/2023_05_22_Pyl-OB_010_small.czi'
# name = "output"

# Usage example:
# czi_file_path = '../../230518_ML_pyl_OB/2023_05_22_Pyl-OB_024.czi'
# output_folder = "2023_05_22_Pyl-OB_024"

czi_file_path = '../../230518_ML_pyl_OB/2023_05_22_Pyl-OB_023.czi'
output_folder = "2023_05_22_Pyl-OB_023"

subpart_size = [1600, 1920]
# process_czi_file(czi_file_path, output_folder,subpart_size)

In [1]:
import multiprocessing

In [8]:
def worker_function(czi_file_path, output_folder,subpart_size):
    try:
        print(czi_file_path)
        print(output_folder)
        print(subpart_size)
        return 1
    except Exception as e:
        print("Exception in worker_function:", e)
        return 0

num_processes = 4  # Number of parallel processes

# Create some data to be processed
input_data1 = ['path1', 'path2', 'path3', 'path4', 'path1', 'path2', 'path3', 'path4']
input_data2 = ['name1', 'name2', 'name3', 'name4', 'name1', 'name2', 'name3', 'name4']
input_data3 = [0, 1, 2, 3, 4, 5, 6, 7]

# Split the input data into chunks for each process
chunk_size = len(input_data1) // num_processes
data_chunks1 = [input_data1[i:i + chunk_size] for i in range(0, len(input_data1), chunk_size)]
data_chunks2 = [input_data2[i:i + chunk_size] for i in range(0, len(input_data2), chunk_size)]
data_chunks3 = [input_data3[i:i + chunk_size] for i in range(0, len(input_data3), chunk_size)]



In [ ]:
# Create a pool of worker processes
print(f"Create a pool of {num_processes} worker processes")
pool = multiprocessing.Pool(processes=num_processes)

try:
    # Distribute the data chunks to worker processes for parallel processing
    print('Distribute the data chunks to worker processes for parallel processing')
    pool.starmap(worker_function, zip(data_chunks1, data_chunks2, data_chunks3))
except Exception as e:
        print("Exception in worker_function:", e)


# Close the pool of worker processes
pool.close()
pool.join()

# Print the final result
print("Final Result")


Create a pool of 4 worker processes
Distribute the data chunks to worker processes for parallel processing


In [ ]:
# import multiprocessing

# def worker_function(data1, data2, data3):
#     result = []  # Store results for this process
#     for item1, item2, item3 in zip(data1, data2, data3):
#         # Perform some computation on the data
#         result.append(item1 * item2 + item3)
#     return result

# if __name__ == "__main__":
#     num_processes = 4  # Number of parallel processes

#     # Create some data to be processed
#     input_data1 = [1, 2, 3, 4, 5, 6, 7, 8]
#     input_data2 = [10, 20, 30, 40, 50, 60, 70, 80]
#     input_data3 = [0, 1, 2, 3, 4, 5, 6, 7]

#     # Split the input data into chunks for each process
#     chunk_size = len(input_data1) // num_processes
#     data_chunks1 = [input_data1[i:i + chunk_size] for i in range(0, len(input_data1), chunk_size)]
#     data_chunks2 = [input_data2[i:i + chunk_size] for i in range(0, len(input_data2), chunk_size)]
#     data_chunks3 = [input_data3[i:i + chunk_size] for i in range(0, len(input_data3), chunk_size)]

#     # Create a pool of worker processes
#     pool = multiprocessing.Pool(processes=num_processes)

#     # Distribute the data chunks to worker processes for parallel processing
#     results = pool.starmap(worker_function, zip(data_chunks1, data_chunks2, data_chunks3))

#     # Combine the results from all processes
#     final_result = []
#     for result in results:
#         final_result.extend(result)

#     # Close the pool of worker processes
#     pool.close()
#     pool.join()

#     # Print the final result
#     print("Final Result:", final_result)
